## Import data & libraries

In [26]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.stats as st
import matplotlib.patches as mpl_patches
import warnings
warnings.filterwarnings("ignore")
from scipy.stats import describe
import matplotlib.cm as cm
import matplotlib.lines as mlines
from pandas.core.window.rolling import Rolling
import pandas_ta as ta

In [27]:
final_df = pd.read_csv('data/final_df.csv')

#como no tenemos muchos datos gratis de git stars y social volume, las quitamos

#final_df.drop(['Social_Volume','Social_Volume_AI','git_stars_day'], axis=1,inplace=True)

In [28]:
final_df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [4]:
#para que ningún nombre de columna tenga espacios
final_df.rename(columns = {'New Addresses':'new_addresses', 'Active Addresses':'active_adresses'}, inplace = True)


In [5]:
final_df.rename(columns = {'Weighted sentiment':'weighted_sentiment'}, inplace = True)

In [29]:
final_df

,Date,opening_price,highest_price,lowest_price,closing_price,transactions_in_blockchain,avg_block_size,sent_by_adress,avg_mining_difficulty,avg_hashrate,...,Bull_Bear_Diff,next_day_closing_price,ISA_9,ISB_26,ITS_9,IKS_26,ICS_26,Social_Volume,Social_Volume_AI,Weighted sentiment
0,2013-01-01,13.5,13.6,13.2,13.3,31734.0,89033.0,26174.0,2.979637e+06,2.433154e+13,...,-4885.43,13.3,16.725,21.6,13.55,16.20,17.8,212,541,-1.919762
1,2013-01-02,13.3,13.4,13.2,13.3,39280.0,114077.0,31809.0,2.979637e+06,2.280465e+13,...,7947.75,13.4,16.725,21.6,13.55,16.20,18.7,310,644,-1.919762
2,2013-01-03,13.3,13.5,13.3,13.4,42147.0,108023.0,38197.0,2.979637e+06,2.372489e+13,...,-1499.67,13.5,16.725,21.6,13.55,16.20,19.5,334,656,-1.919762
3,2013-01-04,13.4,13.5,13.3,13.5,48436.0,141811.0,34990.0,2.979637e+06,2.260818e+13,...,-28891.87,13.4,16.725,21.6,13.55,16.20,19.7,333,579,-1.919762
4,2013-01-05,13.5,13.6,13.3,13.4,39455.0,118240.0,38008.0,2.979637e+06,2.259070e+13,...,-134.04,13.4,16.725,21.6,13.55,16.20,20.4,282,552,-1.919762
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3371,2022-03-26,44331.0,44818.0,44090.0,44548.0,230870.0,499029.0,440067.0,2.745271e+13,2.203984e+20,...,-463.00,46859.0,39611.950,39370.4,42673.40,41238.15,40482.0,14491,18640,-0.205759
3372,2022-03-27,44542.0,46947.0,44445.0,46859.0,218071.0,595651.0,421737.0,2.745271e+13,1.994591e+20,...,-1072.48,47105.0,39842.625,39370.4,43738.50,42064.55,39709.0,13724,18632,-0.368552
3373,2022-03-28,46859.0,48199.0,46672.0,47105.0,293145.0,683575.0,520476.0,2.745271e+13,1.925705e+20,...,-1470.69,47449.0,39941.050,39370.4,44364.50,42690.55,39418.0,15572,20097,0.243179
3374,2022-03-29,47126.0,48127.0,47029.0,47449.0,286789.0,656320.0,519299.0,2.745271e+13,2.206193e+20,...,-626.26,47075.0,39941.050,39370.4,44364.50,42690.55,39464.0,13815,18943,-0.679319


In [7]:
def feature_smoothening(df,feature_name,smoothening_type,smoothening_range=[3,5,7,10,15,30,60,90],show_plot=False,show_original_Feature_in_plot=True):
    if smoothening_type == 'sma':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type} {j}'] = ta.sma(df[feature_name],j) 

    elif smoothening_type == 'var':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type}{j}'] = ta.variance(df[feature_name],j)

    elif smoothening_type == 'stdev':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type}{j}'] = ta.stdev(df[feature_name],j)
    
    elif smoothening_type == 'ema':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type}{j}'] = ta.ema(df[feature_name],j)

    elif smoothening_type == 'wma':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type}{j}'] = ta.wma(df[feature_name],j)

    elif smoothening_type == 'rsi':
        for j in smoothening_range:
             df[f'{feature_name} {smoothening_type}{j}'] = ta.rsi(df[feature_name],j)

    elif smoothening_type == 'roc':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type}{j}'] = ta.roc(df[feature_name],j)  

    elif smoothening_type == 'dema':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type}{j}'] = ta.dema(df[feature_name],j) 

    elif smoothening_type == 'tema':
        for j in smoothening_range:
            df[f'{feature_name} {smoothening_type}{j}'] = ta.tema(df[feature_name],j) 

    elif smoothening_type == 'bband_lower':
        for j in smoothening_range:
            bband_df = ta.bbands(df[feature_name],j)
            df[f'{feature_name} {smoothening_type}{j}'] = bband_df[f'BBL_{j}_2.0']

    elif smoothening_type == 'bband_upper':
        for j in smoothening_range:
            bband_df = ta.bbands(df[feature_name],j)
            df[f'{feature_name} {smoothening_type}{j}'] = bband_df[f'BBU_{j}_2.0']

    elif smoothening_type == 'macd':
        macd_df = ta.macd(df[feature_name])
        df[f'{feature_name} hist {smoothening_type}'] = macd_df['MACDh_12_26_9']
        df[f'{feature_name} signal {smoothening_type}'] = macd_df['MACDs_12_26_9']
        df[f'{feature_name} {smoothening_type}'] = macd_df['MACD_12_26_9']


In [8]:
feature_list = [i for i in list(final_df.columns) if i not in ['Date','next_day_closing_price','ISA_9','ISB_26','ITS_9','IKS_26','ICS_26']]

In [9]:
final_df

,Unnamed: 0.1,Date,opening_price,highest_price,lowest_price,closing_price,transactions_in_blockchain,avg_block_size,sent_by_adress,avg_mining_difficulty,...,next_day_closing_price,ISA_9,ISB_26,ITS_9,IKS_26,ICS_26,Social_Volume,Social_Volume_AI,weighted_sentiment,large_trasactions_over_total
0,0,2013-07-17,97.1,100.0,96.2,98.5,40459.0,100827.0,45703.0,2.616288e+07,...,90.1,108.900,109.55,88.35,87.70,106.8,869,1503,-1.919762,0.152036
1,1,2013-07-18,98.5,98.8,86.2,90.1,41009.0,117039.0,45106.0,2.616288e+07,...,92.0,108.200,107.50,89.85,87.70,109.6,939,1748,-1.236765,0.147757
2,2,2013-07-19,90.1,95.2,87.6,92.0,36588.0,96865.0,43518.0,2.616288e+07,...,89.8,108.300,107.50,94.60,87.70,112.6,828,1501,-1.240729,0.169435
3,3,2013-07-20,92.0,93.1,89.3,89.8,33516.0,82983.0,32982.0,2.616288e+07,...,92.0,108.575,112.00,95.20,87.70,110.0,717,1311,-1.738634,0.089742
4,4,2013-07-21,89.8,92.0,88.0,92.0,35257.0,82978.0,35799.0,2.616288e+07,...,91.6,108.000,112.00,94.05,85.45,109.0,738,1361,-1.169890,0.096192
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,3169,2022-03-21,41282.0,41532.0,40530.0,41022.0,259759.0,612943.0,477545.0,2.745271e+13,...,42373.0,40793.675,40270.80,39994.70,39825.80,40382.0,12950,17680,-0.487291,0.968668
3170,3170,2022-03-22,41022.0,43327.9,40893.0,42373.0,264420.0,639844.0,499333.0,2.745271e+13,...,42912.0,39755.725,40245.50,40470.75,41178.25,39703.0,12919,17689,-0.643219,0.968107
3171,3171,2022-03-23,42373.0,43027.0,41795.0,42912.0,264037.0,702531.0,492109.0,2.745271e+13,...,44013.0,39659.275,40002.50,40774.40,41178.25,40803.0,11380,16913,-1.587868,0.960931
3172,3172,2022-03-24,42911.0,44251.0,42658.0,44013.0,262844.0,661257.0,498050.0,2.745271e+13,...,44331.0,38851.375,39370.40,41602.10,41178.25,41503.0,12317,18027,-1.238464,0.973476


In [10]:
column_1 = final_df["closing_price"]
column_2 = final_df["next_day_closing_price"]
correlation = column_1.corr(column_2)

print(correlation)

0.9988903149878884


## Simple Moving Average
* A simple moving average (SMA) calculates the average of a selected range of feature values for a number of periods in that range.

* It determins if an asset price will continue or if it will reverse a bull or bear trend.

In [11]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'sma',show_plot=True)

## Weighted Moving Average
* While calculating average, It assigns a greater weighting to the most recent data points, and less weighting to data points in the distant past.
![](https://www.thebalance.com/thmb/2bdOUTOzQ0qbp0Y-1ZO_AITMVeg=/2364x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/WMA-2cada5373e76459a947b8c7fc106f41e.jpg)

In [12]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'wma',show_plot=True)

## Exponential Moving Average
* The EMA is a moving average that places a greater weight and significance on the most recent data points. They work similar to WMA but formula is different.
![Image](https://www.thebalance.com/thmb/bUl41pugmmQnuoLXScHzSAAla3c=/1648x0/filters:no_upscale():max_bytes(150000):strip_icc():format(webp)/EMA-a566789cf300425c80352fb70a6c0666.jpg)
* The EMA adapts more quickly to price changes than the SMA does. For example, when a price reverses direction, the EMA will reverse direction more quickly than the SMA will, because the EMA formula gives more weight to recent prices and less weight to prices from the past.


In [13]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'ema',show_plot=True)

## Double Exponential Moving Average
* DEMA responds more quickly to near-term price changes than a normal exponential moving average (EMA). 
 
 ![image](https://wikimedia.org/api/rest_v1/media/math/render/svg/301d4b6f039d7c29ea9d5bf4441e533cd50d72c7)

* It helps to filter out noise

In [14]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'dema',show_plot=True)

## Triple Exponential Moving Average
* It uses multiple EMA calculations and subtracts the lag to create a trend following indicator that reacts quickly to price changes.
![](https://wikimedia.org/api/rest_v1/media/math/render/svg/172c4c79a35109d293d720ef2865581c632800d4)

* The TEMA reacts to price changes quicker than a traditional MA or EMA will. This is because some of the lag has been subtracted out in the calculation.

In [15]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'tema',show_plot=True)

## Standard Deviaiton
* Standard deviation is the statistical measure of market volatility, measuring how widely feature values are dispersed from the average feature values. 
* If feature values trade in a narrow trading range, the standard deviation will return a low value that indicates low volatility. 
* Conversely, if feature values swing wildly up and down, then standard deviation returns a high value that indicates high volatility.

In [16]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'stdev',show_plot=True)

## Variance
* Variance is another statistical measure of market volatility, measuring how widely feature values are dispersed from the average feature values. 
* It is interpreted similar to standard deviation as varaince is square of standadrd deviation

In [17]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'var',show_plot=True)

## Relative Strength Index

> RSI = 100 – 100 / (1 + RS)

> RS = Average of X periods closes up / Average of X periods closes down

An asset is considered oversold or undervalued when the RSI drops below 30. On the other hand, it's deemed to be overbought if the RSI goes above 70.

In [18]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'rsi',show_plot=True,show_original_Feature_in_plot=False)

## Rate of Change
* Measures the percentage change in price between the current feature value and the feature a certain number of periods ago.e.g., 7,30,90

*   A rising ROC above zero typically confirms an uptrend while a falling ROC below zero indicates a downtrend.






In [19]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'roc',show_plot=True,show_original_Feature_in_plot=False)

## Bollinger Bands
* Bollinger Bands are envelopes (Upper and Lower range levels) plotted at a standard deviation level above and below a simple moving average of the price. Because the distance of the bands is based on standard deviation, they adjust to volatility swings in the underlying price.


```
Upper band: n-day SMA + (n-day standard deviation x2)
Lower band: n-day SMA – (n-day standard deviation x2)

```

* Bollinger bands help determine whether values are high or low on a relative basis. They are used in pairs, both upper and lower bands and in conjunction with a moving average. Further, the pair of bands is not intended to be used on its own. Use the pair to confirm signals given with other indicators.


In [20]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'bband_lower',show_plot=True)

In [21]:
for feature in feature_list:
    if feature not in ['Date','weekday', 'year', 'month','7th_day_avg_price','30th_day_avg_price','90th_day_avg_price','avg_btc_price']:
        feature_smoothening(final_df,feature,'bband_upper',show_plot=True)

## Moving Average Convergence Divergence
The MACD represents a trend following indicator that highlights whether the short-term price momentum is moving in the same direction as the long-term price momentum, and in cases where it's not, then it's used to determine if a trend change is near. The MACD consists of four components.

MACD line- shows the variation between the slow moving average and the fast-moving average

Signal line-for signaling fluctuations in price momentum

Histogram- indicates the difference between the signal line and the MACD line.



In [22]:
for feature in feature_list:
    feature_smoothening(final_df,feature,'macd',show_plot=True,show_original_Feature_in_plot=False)

## Final Processing and saving file

In [23]:
final_df = final_df[(final_df['Date'] >= '2013-07-17')]

In [24]:
final_df

,Unnamed: 0.1,Date,opening_price,highest_price,lowest_price,closing_price,transactions_in_blockchain,avg_block_size,sent_by_adress,avg_mining_difficulty,...,Social_Volume macd,Social_Volume_AI hist macd,Social_Volume_AI signal macd,Social_Volume_AI macd,weighted_sentiment hist macd,weighted_sentiment signal macd,weighted_sentiment macd,large_trasactions_over_total hist macd,large_trasactions_over_total signal macd,large_trasactions_over_total macd
0,0,2013-07-17,97.1,100.0,96.2,98.5,40459.0,100827.0,45703.0,2.616288e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2013-07-18,98.5,98.8,86.2,90.1,41009.0,117039.0,45106.0,2.616288e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2013-07-19,90.1,95.2,87.6,92.0,36588.0,96865.0,43518.0,2.616288e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2013-07-20,92.0,93.1,89.3,89.8,33516.0,82983.0,32982.0,2.616288e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,2013-07-21,89.8,92.0,88.0,92.0,35257.0,82978.0,35799.0,2.616288e+07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3169,3169,2022-03-21,41282.0,41532.0,40530.0,41022.0,259759.0,612943.0,477545.0,2.745271e+13,...,-326.996416,-67.143663,138.241489,71.097826,0.020294,0.031211,0.051504,0.001556,-0.002289,-0.000734
3170,3170,2022-03-22,41022.0,43327.9,40893.0,42373.0,264420.0,639844.0,499333.0,2.745271e+13,...,-382.449690,-140.877313,103.022161,-37.855152,-0.015317,0.027381,0.012064,0.002116,-0.001760,0.000356
3171,3171,2022-03-23,42373.0,43027.0,41795.0,42912.0,264037.0,702531.0,492109.0,2.745271e+13,...,-544.306817,-230.168855,45.479947,-184.688908,-0.097370,0.003039,-0.094331,0.001915,-0.001281,0.000634
3172,3172,2022-03-24,42911.0,44251.0,42658.0,44013.0,262844.0,661257.0,498050.0,2.745271e+13,...,-590.168590,-203.390986,-5.367800,-208.758786,-0.121424,-0.027317,-0.148741,0.002501,-0.000656,0.001845


In [ ]:
final_df.to_csv('data/multi_df.csv')